In [2]:
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

In [3]:
import requests
import pprint

#인증키 입력
encoding = 'qUjfGRDJasifRNhDiZxLmTm%2FBUOKOgYdzyBk1RRcOg9EGlFjE3KfP2ABf6exHYkmgLoqi6IsDSsOfXuPd0bWsg%3D%3D'
decoding = 'qUjfGRDJasifRNhDiZxLmTm/BUOKOgYdzyBk1RRcOg9EGlFjE3KfP2ABf6exHYkmgLoqi6IsDSsOfXuPd0bWsg=='

url = 'http://openapi.q-net.or.kr/api/service/rest/InquiryAttenQualSVC/getList?'
params ={'serviceKey' : decoding,
        'pageNo':'1',
        'numOfRows':'600'}

response = requests.get(url, params=params)

contents = response.text

pp = pprint.PrettyPrinter(indent = 4)

xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
rows = xml_obj.findAll('item')

row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

recomJm_df = pd.DataFrame(row_list, columns=name_list)
recomJm_df

,attenJmCd,jmNm,modifyDttm,recomJmCd1,recomJmCd2,recomJmNm1,recomJmNm2,regDttm
0,0011,용접기술사,20231207003240,3100,1022,용접기능장,용접기사,20231207003240
1,0012,금형기술사,20231215,0872,3061,기계기술사,금형제작기능장,20230902230727
2,0050,차량기술사,20231213085614,1027,3110,그린전동자동차기사,자동차정비기능장,20231213085614
3,0060,산업기계설비기술사,20231010095941,0080,0872,건설기계기술사,기계기술사,20231010095941
4,0071,공조냉동기계기술사,20231214193151,0501,1730,건축기계설비기술사,공조냉동기계기사,20231214193151
...,...,...,...,...,...,...,...,...
582,9757,기술지도사(생명공학),20190526004240,0720,1988,화공안전기술사,생물분류기사(식물),20190526004240
583,9758,기술지도사(기술혁신관리),20230913,1431,0872,산업안전기사,기계기술사,20230828103018
584,9759,기술지도사(정보기술관리),20231127,0601,1320,정보관리기술사,정보처리기사,20231127213546
585,9762,수산물품질관리사,20230830154233,9695,1625,농산물품질관리사,수산양식기사,20230830154233


In [5]:
#2-a

import json

# 데이터프레임에서 'recomJm_df'을 가져옴
data_for_finetuning = recomJm_df

# 대화 데이터를 저장할 JSONL 파일
jsonl_file_path = '2번질문_a.jsonl'

# 데이터프레임을 순회하며 GPT-3.5 파인튜닝 데이터로 변환하고 JSONL 파일에 저장
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for index, row in data_for_finetuning.iterrows():
        user_message = f"사용자: {row['jmNm']}랑 비슷한 자격증 알려줘"
        bot_message = f"봇: {row['jmNm']}와 비슷한 자격증은 {row['recomJmNm1']}와 {row['recomJmNm2']}가 있습니다."
        
        # GPT-3.5 파인튜닝 데이터 형식에 맞게 변환
        finetuning_data = {
            "messages": [
                {"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": bot_message}
            ]
        }

        # JSONL 파일에 한 줄씩 저장
        jsonl_file.write(json.dumps(finetuning_data, ensure_ascii=False) + '\n')

# JSONL 파일 출력
with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
    print(jsonl_file.read())

{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 용접기술사랑 비슷한 자격증 알려줘"}, {"role": "assistant", "content": "봇: 용접기술사와 비슷한 자격증은 용접기능장와 용접기사가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 금형기술사랑 비슷한 자격증 알려줘"}, {"role": "assistant", "content": "봇: 금형기술사와 비슷한 자격증은 기계기술사와 금형제작기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 차량기술사랑 비슷한 자격증 알려줘"}, {"role": "assistant", "content": "봇: 차량기술사와 비슷한 자격증은 그린전동자동차기사와 자동차정비기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "use

In [6]:
#2-b

import json

# 데이터프레임에서 'recomJm_df'을 가져옴
data_for_finetuning = recomJm_df

# 대화 데이터를 저장할 JSONL 파일
jsonl_file_path = '2번질문_b.jsonl'

# 데이터프레임을 순회하며 GPT-3.5 파인튜닝 데이터로 변환하고 JSONL 파일에 저장
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for index, row in data_for_finetuning.iterrows():
        user_message = f"사용자: {row['jmNm']}와 유사한 자격증 알려줘"
        bot_message = f"봇: {row['jmNm']}와 비슷한 자격증은 {row['recomJmNm1']}와 {row['recomJmNm2']}가 있습니다."
        
        # GPT-3.5 파인튜닝 데이터 형식에 맞게 변환
        finetuning_data = {
            "messages": [
                {"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": bot_message}
            ]
        }

        # JSONL 파일에 한 줄씩 저장
        jsonl_file.write(json.dumps(finetuning_data, ensure_ascii=False) + '\n')

# JSONL 파일 출력
with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
    print(jsonl_file.read())

{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 용접기술사와 유사한 자격증 알려줘"}, {"role": "assistant", "content": "봇: 용접기술사와 비슷한 자격증은 용접기능장와 용접기사가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 금형기술사와 유사한 자격증 알려줘"}, {"role": "assistant", "content": "봇: 금형기술사와 비슷한 자격증은 기계기술사와 금형제작기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 차량기술사와 유사한 자격증 알려줘"}, {"role": "assistant", "content": "봇: 차량기술사와 비슷한 자격증은 그린전동자동차기사와 자동차정비기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "use

In [7]:
#2-c

import json

# 데이터프레임에서 'recomJm_df'을 가져옴
data_for_finetuning = recomJm_df

# 대화 데이터를 저장할 JSONL 파일
jsonl_file_path = '2번질문_c.jsonl'

# 데이터프레임을 순회하며 GPT-3.5 파인튜닝 데이터로 변환하고 JSONL 파일에 저장
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for index, row in data_for_finetuning.iterrows():
        user_message = f"사용자: {row['jmNm']} 말고 딸 수 있는 비슷한 분야의 자격증이 있을까?"
        bot_message = f"봇: {row['jmNm']}와 비슷한 자격증은 {row['recomJmNm1']}와 {row['recomJmNm2']}가 있습니다."
        
        # GPT-3.5 파인튜닝 데이터 형식에 맞게 변환
        finetuning_data = {
            "messages": [
                {"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": bot_message}
            ]
        }

        # JSONL 파일에 한 줄씩 저장
        jsonl_file.write(json.dumps(finetuning_data, ensure_ascii=False) + '\n')

# JSONL 파일 출력
with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
    print(jsonl_file.read())

{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 용접기술사 말고 딸 수 있는 비슷한 분야의 자격증이 있을까?"}, {"role": "assistant", "content": "봇: 용접기술사와 비슷한 자격증은 용접기능장와 용접기사가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 금형기술사 말고 딸 수 있는 비슷한 분야의 자격증이 있을까?"}, {"role": "assistant", "content": "봇: 금형기술사와 비슷한 자격증은 기계기술사와 금형제작기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 차량기술사 말고 딸 수 있는 비슷한 분야의 자격증이 있을까?"}, {"role": "assistant", "content": "봇: 차량기술사와 비슷한 자격증은 그린전동자동차기사와 자동차정비기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이

In [8]:
#2-d

import json

# 데이터프레임에서 'recomJm_df'을 가져옴
data_for_finetuning = recomJm_df

# 대화 데이터를 저장할 JSONL 파일
jsonl_file_path = '2번질문_d.jsonl'

# 데이터프레임을 순회하며 GPT-3.5 파인튜닝 데이터로 변환하고 JSONL 파일에 저장
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for index, row in data_for_finetuning.iterrows():
        user_message = f"사용자: {row['jmNm']} 대신 취득할 수 있는 자격증 알려줘"
        bot_message = f"봇: {row['jmNm']}와 비슷한 자격증은 {row['recomJmNm1']}와 {row['recomJmNm2']}가 있습니다."
        
        # GPT-3.5 파인튜닝 데이터 형식에 맞게 변환
        finetuning_data = {
            "messages": [
                {"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": bot_message}
            ]
        }

        # JSONL 파일에 한 줄씩 저장
        jsonl_file.write(json.dumps(finetuning_data, ensure_ascii=False) + '\n')

# JSONL 파일 출력
with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
    print(jsonl_file.read())

{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 용접기술사 대신 취득할 수 있는 자격증 알려줘"}, {"role": "assistant", "content": "봇: 용접기술사와 비슷한 자격증은 용접기능장와 용접기사가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 금형기술사 대신 취득할 수 있는 자격증 알려줘"}, {"role": "assistant", "content": "봇: 금형기술사와 비슷한 자격증은 기계기술사와 금형제작기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답합니다."}, {"role": "user", "content": "사용자: 차량기술사 대신 취득할 수 있는 자격증 알려줘"}, {"role": "assistant", "content": "봇: 차량기술사와 비슷한 자격증은 그린전동자동차기사와 자동차정비기능장가 있습니다."}]}
{"messages": [{"role": "system", "content": "당신은 존댓말로 공손하게 대답하는 챗봇입니다. 당신은 자격증 정보에 관한 질문에만 정확하고 간결하게 답변합니다. 자격증 이외의 질문에 대해서는 답변할 수 없다고 대답